In [1]:
# Takes as input a square matrix Q, and outputs a primitive vector v such that v.transpose() * Q * v == 0 if such a v exists, and -1 otherwise.
# The vector v is chosen randomly by the following method:
# A matrix Ts with abs(Ts.det()) == 1 is chosen randomly
# Then v is set to qfsolve of Ts.transpose() * Q * Ts, which is a matrix congruent to Q
# Ts * v / gcd(v) is returned
def randomized_qfsolve(Q, random_steps, random_matrix_entry_size):
    from sage.quadratic_forms.qfsolve import qfsolve
    n = Q.dimensions()[0]
    Ts = identity_matrix(n)
    v = None
    counter = 0
    # Sometimes there is a PARI exception for reasons unclear to me; this is why whe while, try, except construction here is needed
    while (v == None) and (counter < 20):
        counter += 1
        for i in range(random_steps):
            T = matrix(ZZ,n,n,lambda i, j: 0 if i < j
                       else (-1)^(int(random()*2)) if i == j
                       else int(random() * 2 * random_matrix_entry_size) - random_matrix_entry_size)
            Ts = Ts * (T if random() > .5 else T.transpose())
        try:
            v = qfsolve(Ts.transpose() * Q * Ts)
        except:
            v = None
    # qfsolve returns an integer if no solution exists
    if (v == None) or (type(v) == sage.rings.integer.Integer): return -1
    v = v / gcd(v)
    return Ts * v
    
# Takes as input an integer square matrix A that is the Seifert matrix of a knot or link.
# Returns a list [n, T], where n is a non-negative integer and T is a matrix of same size as A with abs(T.det()) == 1
# such that the 2n x 2n upper left submatrix of T * A * T.transpose() is a Seifert matrix of a knot with Alexander polynomial 1
# The function tries to find such an n as large as possible.
# The returned n is a lower bound for g(A) - g_alg(A), where g(A) is the genus of the Seifert surface with Seifert matrix A, i.e. g(A) = rank(A - A.transpose()) / 2,
# and g_alg(A) is defined in Peter Feller, Lukas Lewark: "On classical upper bounds for slice genera"
def galg(A, random_steps = 10, random_matrix_entry_size = 10):
    n = A.dimensions()[0]
    # The algorithm tries to apply a congruence creating a 2 x 2 submatrix with Alexander polynomial 1.
    # If n < 2, no such submatrix can exist. If n == 2, such a submatrix must be the whole matrix, so we just check if the whole matrix has Alexander polynomial 1.
    if (n < 2):
        return [0, identity_matrix(n)]
    if (n == 2):
        if ((A - A.transpose()).det() == 1) and ((A + A.transpose()).det() == 1):
            return [1, identity_matrix(2)]
        return [0, identity_matrix(2)]
    # Find isotropic vector v
    v = randomized_qfsolve(A + A.transpose(), random_steps, random_matrix_entry_size);
    if (v in [-1,-2]):
        return [0, identity_matrix(n)]
    # Complete v to a basis, so that A1[0,0] == 0
    T1 = matrix(ZZ, v).smith_form()[2].inverse();
    A1 = T1 * A * T1.transpose();
    # Apply Euclid's algorithm to first row ignoring the first column, so that the first row of A2 is [0,1,0,...,0], if possible (this may fail)
    v1 = A1[0,range(1,n)]
    X, Y, Z = matrix(ZZ, v1).smith_form()
    c = matrix(ZZ, [1 if i == 0 else 0 for i in range(n-1)])
    if (X != c):
        return [0, identity_matrix(n)]
    T2 = block_matrix([[identity_matrix(1),0],[0,Z.transpose()]])
    A2 = T2 * A1 * T2.transpose();
    # Apply Euclid's algorithm to the first column ignoring the first two rows, so that the first column of A3 is [0,x,y,0,...,0]
    v2 = A2[range(2,n),0]
    T3 = block_matrix([[identity_matrix(2),0],[0,matrix(ZZ, v2).smith_form()[1]]])
    A3 = T3 * A2 * T3.transpose();
    # If y divides x (see previos comment), change the first column so that the first column of A4 is [0,0,y,0,...,0]
    if (A3[1,0] != 0) and ((A3[2,0] == 0) or (A3[1,0] % A3[2,0] != 0)):
        return [0, identity_matrix(n)]
    T4 = identity_matrix(n)
    if (A3[1,0] != 0):
        T4[1,2] = -A3[1,0] // A3[2,0];
    A4 = T4 * A3 * T4.transpose();
    # Use the 1 at (0,1) to clear out the second column, so that the second column of A5 is [1,0,...,0]
    T5 = matrix(ZZ, n, n, lambda i, j: 1 if i == j else -A4[i,1] if j == 0 else 0)
    A5 = T5 * A4 * T4.transpose();
    
    # Continue recursively in the submatrix obtained by deleting the first three (!) rows and columns  
    r = galg(A5[range(3,n),range(3,n)])
    T6 = block_matrix([[identity_matrix(3),0],[0,r[1]]])
    # Use T7 to permute basis vectors, sending the third basis vector to the end
    T7 = matrix(ZZ,n,n,lambda i,j: 1 if ([i,j] in [[0,0],[1,1],[n-1,2]] + [[k-1,k] for k in range(3,n)]) else 0)
    fullT = T7 * T6 * T5 * T4 * T3 * T2 * T1;
    # Now fullT * A * tullT.transpose() has a 2(1 + r[0]) x 2(1 + r[0]) upper left submatrix that looks like this (in case of 4 x 4):
    # 0 1 * *
    # 0 0 * *
    # 0 0 0 1 
    # 0 0 0 0
    # and similarly for larger matrices. * signifies arbitrary elements. This submatrix has Alexander polynomial 1.
    return [1 + r[0], fullT]